# Geodatenanalyse 2: Termin 2 - Datenaufbereitung
# Aufgabe

Bereite jetzt selbst den Datensatz "Nitratmessungen_aufgabe.csv" auf:
- [x] Ersetze die Werte unter der NWG 
- [x] Ersetze alle NULL-Values (bzw. Platzhalter für NaN)
- [x] Aggregiere die Daten für jede Messstelle
- [x] Splitte die Date (85:15)
- [x] Standardisiere die numerischen Variablen
- [x] Encode die Landnutzung binär nach der Landwirtschaft (Corine = 2XX) --> vorhanden = 1, nicht vorhanden = 0
- [x] Target-Encode die Hyrogeologie auf Basis der Nitratkonzentration
- [x] Speichere deine Ergebnisse
- [x] Fertig! :)

Tipps:
- für die Features der Sauerstoff-Konzentration und der Hydrogeologie liegen die NULL-Values nicht als diese direkt vor, hier ist es hilfreich mal die Features mit ".describe()" bzw. ".unique()" zu betrachten

#### Laden der Bibliotheken

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


#### Nitratdatensatz:

In [2]:
#Mögliche Fehler: encoding falsch, sep falsch, index_columns falsch gewählt 
data1 = pd.read_csv('Nitratmessungen_aufgabe.csv', sep=';', encoding="latin1")
data1.head()

,Messstelle,GW-Nummer,Datum,NO3 [mg/l],O2 [mg/l],RASTERVALU,HYDROGEOL3
0,"BBR Betonwerk, Umkirch",918/069-2,11.10.2006 15:20,"9,4","2,4",121,Quartäre Kiese und Sande (GWL)
1,"BBR Betonwerk, Umkirch",918/069-2,17.09.2007 14:20,12,"2,1",121,Quartäre Kiese und Sande (GWL)
2,"BBR Betonwerk, Umkirch",918/069-2,23.09.2008 08:05,"7,6","2,5",121,Quartäre Kiese und Sande (GWL)
3,"BBR Betonwerk, Umkirch",918/069-2,10.11.2008 09:10,"12,2","1,2",121,Quartäre Kiese und Sande (GWL)
4,"BBR Betonwerk, Umkirch",918/069-2,09.09.2009 14:08,12,"1,3",121,Quartäre Kiese und Sande (GWL)


#### Filtern des Datensatzes

In [3]:
data = data1[['Messstelle','GW-Nummer','Datum', 'NO3 [mg/l]','O2 [mg/l]','RASTERVALU', 'HYDROGEOL3']]
data = data.assign(Datum=pd.to_datetime(data['Datum'], format='%d.%m.%Y %H:%M'))
data.head()

,Messstelle,GW-Nummer,Datum,NO3 [mg/l],O2 [mg/l],RASTERVALU,HYDROGEOL3
0,"BBR Betonwerk, Umkirch",918/069-2,2006-10-11 15:20:00,"9,4","2,4",121,Quartäre Kiese und Sande (GWL)
1,"BBR Betonwerk, Umkirch",918/069-2,2007-09-17 14:20:00,12,"2,1",121,Quartäre Kiese und Sande (GWL)
2,"BBR Betonwerk, Umkirch",918/069-2,2008-09-23 08:05:00,"7,6","2,5",121,Quartäre Kiese und Sande (GWL)
3,"BBR Betonwerk, Umkirch",918/069-2,2008-11-10 09:10:00,"12,2","1,2",121,Quartäre Kiese und Sande (GWL)
4,"BBR Betonwerk, Umkirch",918/069-2,2009-09-09 14:08:00,12,"1,3",121,Quartäre Kiese und Sande (GWL)


## 1. Ersetze die Werte unter der NWG 

In [4]:
# Funktion definieren um Werte unter der NWG durch 0.5*NWG zu ersetzen
def u_NWG(X):
    if type(X) == str:
        if '<' in X:
            Y = float(X.split('<')[1].replace(',', '.'))/2
        else:
            Y = float(X.replace(',','.'))
    else:
        Y = X
    return float(Y)

In [5]:
# Neue Spalte mit korrigierten 02 und NO3 einfügen und alte Spalten löschen
data = data.assign(O2= data['O2 [mg/l]'].apply(u_NWG),
           NO3 = data['NO3 [mg/l]'].apply(u_NWG))
data.drop(columns=['NO3 [mg/l]','O2 [mg/l]'], inplace=True)
data.sort_values("NO3", ascending=True).head()

,Messstelle,GW-Nummer,Datum,RASTERVALU,HYDROGEOL3,O2,NO3
7987,"GWM 3 Firma CU Chemie Uetikon, Lahr",374/066-7,2007-09-12 11:55:00,121,Quartäre Kiese und Sande (GWL),0.3,0.05
4099,"GWM E21/1 ZWK Kurpfalzwalldorf, Walldorf",22/306-3,2018-03-13 15:50:00,313,Quartäre Kiese und Sande (GWL),0.4,0.05
4614,"GWM Flach (1960), Auenheim",2010/113-0,2007-09-14 13:10:00,211,Quartäre Kiese und Sande (GWL),0.3,0.05
3792,"GWM B2 Flach, Neumühl",2006/114-9,2007-09-14 10:45:00,211,Quartäre Kiese und Sande (GWL),0.3,0.05
2852,"BR 5 Südzucker, Waghäusel",244/307-5,2007-09-20 16:03:00,121,Quartäre Kiese und Sande (GWL),0.1,0.05


## 2. Ersetze alle NULL-Values (bzw. Platzhalter für NaN)

Platzhalter zuerst durch Nan ersetzen und anschließend NaN durch Mittelwert ersetzen.

Ansonsten wird der Mittelwert falsch berechnet.

Platzhalter ist hier -999. Es gibt unterschiedliche Funktionen wie diese Werte identifiziert werden können

In [38]:
data.describe()

,RASTERVALU,O2,NO3
count,12178.000000,12178.000000,12178.000000
mean,205.882165,3.367765,25.870776
std,71.897127,2.858819,26.777008
min,111.000000,0.050000,0.050000
25%,121.000000,0.500000,3.200000
50%,211.000000,2.950000,17.300000
75%,231.000000,5.900000,42.000000
max,512.000000,11.800000,204.300000


In [39]:
print(data['O2'].mean())
print(data['O2'].min())

3.367764675724173
0.05


### 2.1 Ersetze Platzhalter

In [9]:
# -999 mit None ersetzen
data['O2'].replace(-999.0, None, inplace=True)
data['O2'].mean()

3.3677646757241546

In [10]:
data['HYDROGEOL3'].unique()

array(['Quartäre Kiese und Sande (GWL)', 'Oberer Muschelkalk (GWL)',
       'Paläozoikum, Kristallin (GWG)', '-',
       'Unterjura und Mitteljura (GWG)',
       'Tertiär im Oberrheingraben (GWG)',
       'Oberjura (Raurasische Fazies) (GWL)'], dtype=object)

In [11]:
# - mit None ersetzen
data['HYDROGEOL3'].replace("-", None, inplace=True)
data['HYDROGEOL3'].unique()

array(['Quartäre Kiese und Sande (GWL)', 'Oberer Muschelkalk (GWL)',
       'Paläozoikum, Kristallin (GWG)', None,
       'Unterjura und Mitteljura (GWG)',
       'Tertiär im Oberrheingraben (GWG)',
       'Oberjura (Raurasische Fazies) (GWL)'], dtype=object)

### 2.2 Ersetzen von fehlenden Werte (NULL-Values)


Kontrolle der NULL-Values:

In [12]:
data.isnull().sum()

Messstelle      0
GW-Nummer       0
Datum           0
RASTERVALU      0
HYDROGEOL3     56
O2            475
NO3            20
dtype: int64

#### 2.2.1 Nan in numerische Spalten

In [13]:
# NaNs mit Mittelwerten ersetzen
data['O2'].fillna(data.O2.mean(), inplace=True)
data['NO3'].fillna(data.NO3.mean(), inplace=True)
data.head()

,Messstelle,GW-Nummer,Datum,RASTERVALU,HYDROGEOL3,O2,NO3
0,"BBR Betonwerk, Umkirch",918/069-2,2006-10-11 15:20:00,121,Quartäre Kiese und Sande (GWL),2.4,9.4
1,"BBR Betonwerk, Umkirch",918/069-2,2007-09-17 14:20:00,121,Quartäre Kiese und Sande (GWL),2.1,12.0
2,"BBR Betonwerk, Umkirch",918/069-2,2008-09-23 08:05:00,121,Quartäre Kiese und Sande (GWL),2.5,7.6
3,"BBR Betonwerk, Umkirch",918/069-2,2008-11-10 09:10:00,121,Quartäre Kiese und Sande (GWL),1.2,12.2
4,"BBR Betonwerk, Umkirch",918/069-2,2009-09-09 14:08:00,121,Quartäre Kiese und Sande (GWL),1.3,12.0


#### 2.2.1 NAN in kategorische Spalten


In [14]:
# NaNs mit Mode ersetzen
hy_mode = data.HYDROGEOL3.mode()[0]
print(data.HYDROGEOL3.value_counts(),'\n\n',
      'Modus:',
      hy_mode)

Quartäre Kiese und Sande (GWL)         11955
Paläozoikum, Kristallin (GWG)             50
Tertiär im Oberrheingraben (GWG)          37
Oberjura (Raurasische Fazies) (GWL)       35
Oberer Muschelkalk (GWL)                  26
Unterjura und Mitteljura (GWG)            19
Name: HYDROGEOL3, dtype: int64 

 Modus: Quartäre Kiese und Sande (GWL)


In [15]:
data = data.assign(Hydrogeologie = data.loc[:,'HYDROGEOL3'].fillna(hy_mode))
data.drop(columns='HYDROGEOL3', inplace=True)

#### Kontrolle

In [16]:
data.isnull().sum()

Messstelle       0
GW-Nummer        0
Datum            0
RASTERVALU       0
O2               0
NO3              0
Hydrogeologie    0
dtype: int64

## 3. Aggregiere die Daten für jede Messstelle

In [17]:
# Nach Messstellen Gruppieren und Mittelwerte bzw. Modes berechnen
messstellen1 = data[['Messstelle','NO3','O2']].groupby('Messstelle').mean()
messstellen2 = data[['Messstelle','Hydrogeologie','RASTERVALU']].groupby('Messstelle').agg(pd.Series.mode)

messstellen1.reset_index(inplace=True)
messstellen2.reset_index(inplace=True)

In [18]:
# Datframe zusammenfügen
Messstellen  = messstellen1.merge(messstellen2, on='Messstelle')
Messstellen.head()

,Messstelle,NO3,O2,Hydrogeologie,RASTERVALU
0,"BBR 1 Firma Schultis, Riegel",6.259259,1.315473,Quartäre Kiese und Sande (GWL),121
1,"BBR 2 Firma Thieme, Teningen",1.452941,0.823723,Quartäre Kiese und Sande (GWL),121
2,"BBR 2 Kronenwiese Firma Burda Werk 1, Offenburg",5.600000,1.984615,Quartäre Kiese und Sande (GWL),121
3,"BBR 3186 im Garten der alten Schule, Langhurst",6.968750,0.730346,Quartäre Kiese und Sande (GWL),112
4,"BBR 998 A Kehlerstrasse, Neuried-Auenheim",56.164286,1.116071,Quartäre Kiese und Sande (GWL),112


## 4. Splitte die Date (85:15)

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
# Train Test Split 85/15
train, test = train_test_split(Messstellen, test_size=0.15, random_state=43)

In [42]:
print('Größe Trainingsdaten: ', train.shape)
print('Größe Testdaten: ', test.shape)
print('Größe Testdaten in %: ',(len(test)/len(train)*100))

Größe Trainingsdaten:  (430, 9)
Größe Testdaten:  (77, 9)
Größe Testdaten in %:  17.906976744186046


## 5. Standardisiere die numerischen Variablen

### 5.1 Standardisieren

In [26]:
from sklearn.preprocessing import StandardScaler

In [27]:
# scaler Objekt definieren und an train NO3 anwenden
scaler = StandardScaler()
scaler.fit(train[['NO3']])

StandardScaler()

In [28]:
# Train und Test set standardisieren
train['NO3_std']= scaler.transform(train[['NO3']])
test['NO3_std'] = scaler.transform(test[['NO3']])

In [44]:
print('mean(X): ', train.NO3_std.mean().round(2))
print('std(X): ', train.NO3_std.std().round(2))

mean(X):  0.0
std(X):  1.0


In [30]:
train.head()

,Messstelle,NO3,O2,Hydrogeologie,RASTERVALU,NO3_norm,NO3_std
238,"GWM 3661 Friedhof, Wiesental",60.560000,0.980000,Quartäre Kiese und Sande (GWL),111,0.487735,1.537763
102,"BR NB 38 Main/Feldbergstrasse, KA-Weiherfeld",24.163636,0.678888,Quartäre Kiese und Sande (GWL),112,0.193532,0.048512
463,"TB Firma Fischerwerke GmbH, Emmendingen",19.300000,4.252961,Quartäre Kiese und Sande (GWL),121,0.154218,-0.150496
165,"GWM 1138 A, Mahlberg",7.272000,3.692711,Quartäre Kiese und Sande (GWL),211,0.056992,-0.642652
440,"TB 2 neu, Auggen",34.003448,6.668467,Tertiär im Oberrheingraben (GWG),211,0.273070,0.451134


In [31]:
test.head()

,Messstelle,NO3,O2,Hydrogeologie,RASTERVALU,NO3_norm,NO3_std
498,"TB WW Neuburgweier, Rheinstetten",0.664062,1.625000,Quartäre Kiese und Sande (GWL),231,0.003578,-0.913033
328,"GWM K 20 Kehl, Goldscheuer",0.227778,0.255556,Quartäre Kiese und Sande (GWL),231,0.000051,-0.930885
321,"GWM G0/1 KIT, Leopoldshafen",6.945455,3.259091,Quartäre Kiese und Sande (GWL),313,0.054352,-0.656014
170,"GWM 1279 A, Oberhausen",14.138462,7.215385,Quartäre Kiese und Sande (GWL),231,0.112496,-0.361694
432,"TB 116 Universität Albertstrasse, Freiburg",14.133333,7.412369,Quartäre Kiese und Sande (GWL),121,0.112454,-0.361903


## 6. Encode die Landnutzung binär nach der Landwirtschaft (Corine = 2XX) &rarr; vorhanden = 1, nicht vorhanden = 0

In [32]:
# Encodieren
train['Corine'] = train['RASTERVALU'].apply(lambda x: int(str(x)[0]))
train['Artificial_Surface'] = train['Corine'].apply(lambda x: 1 if x == 2  else 0)

test['Corine'] = test['RASTERVALU'].apply(lambda x: int(str(x)[0]))
test['Artificial_Surface'] = test['Corine'].apply(lambda x: 1 if x == 2  else 0)

In [33]:
# Nicht mehr benötigte Spalte entfernen
train.drop(columns='RASTERVALU', inplace=True)
test.drop(columns='RASTERVALU', inplace=True)

In [34]:
train

,Messstelle,NO3,O2,Hydrogeologie,NO3_norm,NO3_std,Corine,Artificial_Surface
238,"GWM 3661 Friedhof, Wiesental",60.560000,0.980000,Quartäre Kiese und Sande (GWL),0.487735,1.537763,1,0
102,"BR NB 38 Main/Feldbergstrasse, KA-Weiherfeld",24.163636,0.678888,Quartäre Kiese und Sande (GWL),0.193532,0.048512,1,0
463,"TB Firma Fischerwerke GmbH, Emmendingen",19.300000,4.252961,Quartäre Kiese und Sande (GWL),0.154218,-0.150496,1,0
165,"GWM 1138 A, Mahlberg",7.272000,3.692711,Quartäre Kiese und Sande (GWL),0.056992,-0.642652,2,1
440,"TB 2 neu, Auggen",34.003448,6.668467,Tertiär im Oberrheingraben (GWG),0.273070,0.451134,2,1
...,...,...,...,...,...,...,...,...
277,"GWM B2 Tief Kieswerk Peter, Honau",0.958333,1.008333,Quartäre Kiese und Sande (GWL),0.005957,-0.900992,5,0
305,"GWM F Sportzentrum, Schriesheim",5.061667,5.246667,Quartäre Kiese und Sande (GWL),0.039125,-0.733094,1,0
255,"GWM 6, Neuried Dundenheim",74.896429,3.650118,Quartäre Kiese und Sande (GWL),0.603620,2.124375,2,1
320,"GWM Friedrichstal, Friedrichstal",4.788462,1.014271,Quartäre Kiese und Sande (GWL),0.036917,-0.744273,2,1


## 7. Target-Encode die Hyrogeologie auf Basis der Nitratkonzentration

In [35]:
from category_encoders.target_encoder import TargetEncoder

In [36]:
# Target (Mittelwert)- Encoding
encoder = TargetEncoder()
train['Target_Hy'] = encoder.fit_transform(train['Hydrogeologie'],train['NO3'])
test['Target_Hy'] = encoder.transform(test['Hydrogeologie'])
train.head()

,Messstelle,NO3,O2,Hydrogeologie,NO3_norm,NO3_std,Corine,Artificial_Surface,Target_Hy
238,"GWM 3661 Friedhof, Wiesental",60.560000,0.980000,Quartäre Kiese und Sande (GWL),0.487735,1.537763,1,0,23.047591
102,"BR NB 38 Main/Feldbergstrasse, KA-Weiherfeld",24.163636,0.678888,Quartäre Kiese und Sande (GWL),0.193532,0.048512,1,0,23.047591
463,"TB Firma Fischerwerke GmbH, Emmendingen",19.300000,4.252961,Quartäre Kiese und Sande (GWL),0.154218,-0.150496,1,0,23.047591
165,"GWM 1138 A, Mahlberg",7.272000,3.692711,Quartäre Kiese und Sande (GWL),0.056992,-0.642652,2,1,23.047591
440,"TB 2 neu, Auggen",34.003448,6.668467,Tertiär im Oberrheingraben (GWG),0.273070,0.451134,2,1,19.705587


## 8. Speichere deine Ergebnisse

In [37]:
#train.to_csv('Aufg_1_train.csv')
#test.to_csv('Aufg_1_test.csv')

## Fertig